In [5]:
import numpy as np
import depthai as dai
import json
import sys
import os
import tifffile as tiff
import math
import matplotlib.pyplot as plt


os.chdir(r"C:\Users\hnvul\Documents\GitHub\fishsense-oakd\Videos")

frame = tiff.imread('depth.tif', key=range(20))
dispFrame = tiff.imread('disp.tif', key=range(20))
f = open('calibData.json')
data = json.load(f)
monoHFOV = np.deg2rad(data['cameraData'][1][1]['specHfovDeg'])

def _calc_angle(frame, offset):
    return math.atan(math.tan(monoHFOV / 2.0) * offset / (frame.shape[1] / 2.0))

# roi has to be list of ints
def calc_spatials(depthFrame, roi, averaging_method=np.mean):
    # roi = self._check_input(roi, depthFrame) # If point was passed, convert it to ROI
    THRESH_LOW = 200 # 20cm
    THRESH_HIGH = 30000 # 30m

    x, y = roi

    # Calculate the average depth in the ROI.
    depthROI = depthFrame[y, x]
    inRange = (THRESH_LOW <= depthROI) & (depthROI <= THRESH_HIGH)

    averageDepth = averaging_method(depthROI[inRange])

    centroid = { # Get centroid of the ROI
        'x': int((x + x) / 2),
        'y': int((y + y) / 2)
    }

    midW = int(depthFrame.shape[1] / 2) # middle of the depth img width
    midH = int(depthFrame.shape[0] / 2) # middle of the depth img height
    bb_x_pos = centroid['x'] - midW
    bb_y_pos = centroid['y'] - midH

    angle_x = _calc_angle(depthFrame, bb_x_pos)
    angle_y = _calc_angle(depthFrame, bb_y_pos)

    spatials = {
        'z': averageDepth,
        'x': averageDepth * math.tan(angle_x),
        'y': -averageDepth * math.tan(angle_y)
    }
    return spatials, centroid

In [6]:
print(frame.shape)
print(dispFrame.shape)

(20, 400, 640)
(20, 400, 640)


In [ ]:
array = np.empty((frame.shape[0], frame.shape[1] * frame.shape[2],3))
for k in range (0, frame.shape[0]):
    temp = 0
    for i in range(0, frame.shape[1]):
        for j in range(0, frame.shape[2]):
            coords, pixel = calc_spatials(frame[k], [j,i], averaging_method=np.mean)
            pixelCoords = np.array([coords['x'], coords['y'], coords['z']])
            if (np.isnan(pixelCoords).any()):
                continue
            array[k,temp] = pixelCoords
            temp += 1

C:\Users\hnvul\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hnvul\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
print(array.shape)
print(array)

In [ ]:
import open3d as o3d
# geometry = o3d.geometry.PointCloud()
# geometry.points = o3d.utility.Vector3dVector(array[18])
# o3d.visualization.draw_geometries([geometry])

vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()
geometry = o3d.geometry.PointCloud()
geometry.points = o3d.utility.Vector3dVector(array[0])
vis.add_geometry(geometry)
for i in range(1,frame.shape[0]):
    geometry.points = o3d.utility.Vector3dVector(array[i])
    vis.update_geometry(geometry)
    vis.poll_events()
    vis.update_renderer()
vis.destroy_window()

# vis = o3d.visualization.Visualizer()
# vis.create_window()
# # geometry is the point cloud used in your animaiton
# geometry = o3d.geometry.PointCloud()
# geometry.points = o3d.utility.Vector3dVector(array[0])
# vis.add_geometry(geometry)

# for i in range(1,frame.shape[0]):
#     # now modify the points of your geometry
#     # you can use whatever method suits you best, this is just an example
#     geometry.points = o3d.utility.Vector3dVector(array[i])
#     vis.update_geometry(geometry)
#     vis.poll_events()
#     vis.update_renderer()
# vis.destroy_window()